In [3]:
import pickle
import numpy as np
import os

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

import open3d as o3d

from collections import defaultdict

In [4]:
%matplotlib inline

In [5]:
%matplotlib qt

In [65]:
output_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/"

# Save input pointcloud

In [ ]:
obj_names = ["005_tomato_soup_can", "035_power_drill", "052_extra_large_clamp", "065-i_cups"]

sel_grasps = {
    "5": ["3"],
    "10": ["3"],
    "50": ["0"]
}

input_pointcloud_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/ycb_different_grasps_2/ycb_con_grasps_2/"

In [82]:
output_pointcloud_dir = os.path.join(output_dir, "input_pointcloud")

for sel_grasp in sel_grasps:
    for grasp_num in sel_grasps[sel_grasp]:
        for obj_name in obj_names:
            input_pointcloud_pth = os.path.join(input_pointcloud_dir, sel_grasp + "_grasps", obj_name + "_" + sel_grasp + "-" + grasp_num, "pointcloud.npz")
            input_pointcloud = o3d.geometry.PointCloud()
            input_pointcloud.points = o3d.utility.Vector3dVector(np.load(input_pointcloud_pth)['points'])
            
            out_pth = os.path.join(output_pointcloud_dir, obj_name + "_" + sel_grasp + "-" + grasp_num + ".ply")
            o3d.io.write_point_cloud(out_pth, input_pointcloud)

# Reconstruct objects

In [133]:
ply_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/input_pointcloud/"
out_BallPivoting_dir = os.path.join(output_dir, "reconstruction_BallPivoting")

if not os.path.exists(out_BallPivoting_dir):
    os.makedirs(out_BallPivoting_dir)
    
out_Poisson_dir = os.path.join(output_dir, "reconstruction_Poisson")
if not os.path.exists(out_Poisson_dir):
    os.makedirs(out_Poisson_dir)

In [147]:
%%capture
for obj_name in os.listdir(ply_dir):
    ply_pth = os.path.join(ply_dir, obj_name)
    
    # Load pointcloud
    pcd_in = o3d.io.read_point_cloud(ply_pth)

    # Estimate normals and orient them
    pcd_in.estimate_normals()
    pcd_in.orient_normals_towards_camera_location(camera_location=np.array([0., 0., 0.]))
    normals = np.asarray(pcd_in.normals)
    pcd_in.normals = o3d.utility.Vector3dVector(- normals)

    # Reconstruct object Ball pivoting
    radii = [0.005, 0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14]

    rec_BP_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd_in, o3d.utility.DoubleVector(radii))
    rec_BP_mesh.paint_uniform_color([0.7, 0.7, 0.7])
    # o3d.visualization.draw_geometries([rec_mesh], mesh_show_back_face=True)

    # Save object Ball pivoting
    out_pth = os.path.join(out_BallPivoting_dir, obj_name)
    o3d.io.write_triangle_mesh(out_pth, rec_BP_mesh)
    
    # Reconstruction Poisson
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        rec_P_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_in, depth=9)
    rec_P_mesh.compute_triangle_normals()
    #rec_P_mesh.compute_vertex_normals()
    rec_P_mesh.paint_uniform_color([0.7, 0.7, 0.7])
    
    # Save object Poisson
    out_pth = os.path.join(out_Poisson_dir, obj_name)
    o3d.io.write_triangle_mesh(out_pth, rec_P_mesh)